# Get the top 100 book title from Guardian
Weichen Fang
13218115

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

# get the top 100 book titles from The Guardian
url = "https://www.theguardian.com/books/2015/aug/17/the-100-best-novels-written-in-english-the-full-list"
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

book_titles = soup.find_all("p", class_="dcr-1a568om")

# Create an empty list
book_list = []

# Extract the book titles and add them to the list
for title in book_titles:
    book_link = title.find("a")
    if book_link is not None:
        book_title = book_link.text.strip()
        book_list.append(book_title)

# Empty lists to store book details
descriptions = []
authors = []
genres = []
links = []
print(book_list)

['The Pilgrim’s Progress', 'Robinson Crusoe', 'Gulliver’s Travels', 'Clarissa', 'Tom Jones', 'The Life and Opinions of Tristram Shandy, Gentleman', 'Emma', 'Frankenstein', 'Nightmare Abbey', 'Thomas Love Peacock', 'The Narrative of Arthur Gordon Pym of Nantucket', 'Sybil', 'Jane Eyre', 'Wuthering Heights', 'Vanity Fair', 'David Copperfield', 'The Scarlet Letter', 'Moby-Dick', 'Alice’s Adventures in Wonderland', 'The Moonstone', 'Little Women', 'Middlemarch', 'The Way We Live Now', 'The Adventures of Huckleberry Finn', 'Kidnapped', 'Three Men in a Boat', 'The Sign of Four', 'The Picture of Dorian Gray', 'New Grub Street', 'Jude the Obscure', 'The Red Badge of Courage', 'Dracula', 'Heart of Darkness', 'Sister Carrie', 'Kim', 'The Call of the Wild', 'The Golden Bowl', 'Hadrian the Seventh', 'The Wind in the Willows', 'The History of Mr Polly', 'Zuleika Dobson', 'The Good Soldier', 'The Thirty-Nine Steps', 'The Rainbow', 'Of Human Bondage', 'The Age of Innocence', 'Ulysses', 'Babbitt', 'A 

# Scrap extra information for each top 100 book in Goodreads
most elegant code I can come up with, but down side is this will running iteration to scrap evry think I want, from experience 4 times is enough to get the full dataframe, however this will take half an hour to run. 
thinking to use ways to optimize this, maybe selenium?

In [2]:
#final version with iteration
from bs4 import BeautifulSoup
# Function to scrape book details
def scrape_book_details(book):
    try:
        # Creating search query for Goodreads
        search_query = book.replace(" ", "+")
        url = f"https://www.goodreads.com/search?q={search_query}"

        # Making HTTP GET request
        response = requests.get(url)
        soup = BeautifulSoup(response.content, "html.parser")

        # Finding the first search result (most relevant) from Goodreads
        search_result = soup.find("a", class_="bookTitle")

        if search_result:
            # Extracting book details if search result is found
            book_url = "https://www.goodreads.com" + search_result["href"]
            book_response = requests.get(book_url)
            book_soup = BeautifulSoup(book_response.content, "html.parser")

            # Extracting book description
            description_element = book_soup.find("div", {"data-testid": "description"})
            description = description_element.find("span", class_="Formatted").text.strip() if description_element else "N/A"

            # Extracting author
            author_element = book_soup.find("span", class_="ContributorLink__name")
            author = author_element.text.strip() if author_element else "N/A"

            # Extracting genres
            genres_element = book_soup.find("div", {"data-testid": "genresList"})
            genre_links = genres_element.find_all("a", class_="Button Button--tag-inline Button--small")
            genre_list = [genre.text.strip() for genre in genre_links] if genre_links else ["N/A"]

            return description, author, book_url, genre_list
        else:
            return "N/A", "N/A", "N/A", ["N/A"]
    except Exception as e:
        # Handle the exception
        print(f"Error occurred while scraping book '{book}': {e}")
        return "N/A", "N/A", "N/A", ["N/A"]

# Select the top n books from the book_list
top_books = book_list

# Initialize lists
descriptions = []
authors = []
links = []
genres = []

# Iterate over the top_books
for book in top_books:
    description, author, link, genre = scrape_book_details(book)
    iteration_count = 0

    # Re-scrape if any value is "N/A" or ["N/A"]
    while (description == "N/A" or author == "N/A" or link == "N/A" or genre == ["N/A"]) and iteration_count < 4:
        print(f"Re-scraping book '{book}' (Iteration {iteration_count + 1})...")
        description, author, link, genre = scrape_book_details(book)
        iteration_count += 1


    descriptions.append(description)
    authors.append(author)
    links.append(link)
    genres.append(genre)


# Create a dictionary with book details
data = {
    "Book Title": top_books,
    "Description": descriptions,
    "Author": authors,
    "Link": links,
    "Genres": genres
}

# Create a DataFrame
df = pd.DataFrame(data)

# Print the DataFrame
print(df)


Error occurred while scraping book 'The Pilgrim’s Progress': 'NoneType' object has no attribute 'find_all'
Re-scraping book 'The Pilgrim’s Progress' (Iteration 1)...
Error occurred while scraping book 'Clarissa': 'NoneType' object has no attribute 'find_all'
Re-scraping book 'Clarissa' (Iteration 1)...
Error occurred while scraping book 'Tom Jones': 'NoneType' object has no attribute 'find_all'
Re-scraping book 'Tom Jones' (Iteration 1)...
Error occurred while scraping book 'Nightmare Abbey': 'NoneType' object has no attribute 'find_all'
Re-scraping book 'Nightmare Abbey' (Iteration 1)...
Error occurred while scraping book 'Nightmare Abbey': 'NoneType' object has no attribute 'find_all'
Re-scraping book 'Nightmare Abbey' (Iteration 2)...
Error occurred while scraping book 'Nightmare Abbey': 'NoneType' object has no attribute 'find_all'
Re-scraping book 'Nightmare Abbey' (Iteration 3)...
Error occurred while scraping book 'Nightmare Abbey': 'NoneType' object has no attribute 'find_all'


                         Book Title  \
0            The Pilgrim’s Progress   
1                   Robinson Crusoe   
2                Gulliver’s Travels   
3                          Clarissa   
4                         Tom Jones   
..                              ...   
97                Breathing Lessons   
98                    Amongst Women   
99                       Underworld   
100                        Disgrace   
101  True History of the Kelly Gang   

                                           Description          Author  \
0    This famous story of man's progress through li...     John Bunyan   
1    Daniel Defoe relates the tale of an English sa...    Daniel Defoe   
2    A wickedly clever satire uses comic inversions...  Jonathan Swift   
3    In a gripping novel perfect for fans of Sadie ...    Ripley Jones   
4    A foundling of mysterious parentage brought up...  Henry Fielding   
..                                                 ...             ...   
97   Maggie a

In [3]:
df.info()
df.head(102)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102 entries, 0 to 101
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Book Title   102 non-null    object
 1   Description  102 non-null    object
 2   Author       102 non-null    object
 3   Link         102 non-null    object
 4   Genres       102 non-null    object
dtypes: object(5)
memory usage: 4.1+ KB


,Book Title,Description,Author,Link,Genres
0,The Pilgrim’s Progress,This famous story of man's progress through li...,John Bunyan,https://www.goodreads.com/book/show/29797.The_...,"[Classics, Fiction, Christian, Religion, Chris..."
1,Robinson Crusoe,Daniel Defoe relates the tale of an English sa...,Daniel Defoe,https://www.goodreads.com/book/show/2932.Robin...,"[Classics, Fiction, Adventure, Literature, His..."
2,Gulliver’s Travels,A wickedly clever satire uses comic inversions...,Jonathan Swift,https://www.goodreads.com/book/show/7733.Gulli...,"[Classics, Fiction, Fantasy, Adventure, Litera..."
3,Clarissa,In a gripping novel perfect for fans of Sadie ...,Ripley Jones,https://www.goodreads.com/book/show/60784571-m...,"[Mystery, Young Adult, Thriller, Mystery Thril..."
4,Tom Jones,A foundling of mysterious parentage brought up...,Henry Fielding,https://www.goodreads.com/book/show/99329.The_...,"[Classics, Fiction, Literature, 18th Century, ..."
...,...,...,...,...,...
97,Breathing Lessons,Maggie and Ira Moran have been married for twe...,Anne Tyler,https://www.goodreads.com/book/show/31181.Brea...,"[Fiction, Literary Fiction, Contemporary, Nove..."
98,Amongst Women,Moran is an old Republican whose life was fore...,John McGahern,https://www.goodreads.com/book/show/304209.Amo...,"[Fiction, Irish Literature, Ireland, Classics,..."
99,Underworld,His powers—Inhuman...His passion—Beyond immort...,Gena Showalter,https://www.goodreads.com/book/show/476543.The...,"[Paranormal, Paranormal Romance, Romance, Fant..."
100,Disgrace,"Set in post-apartheid South Africa, J. M. Coet...",J.M. Coetzee,https://www.goodreads.com/book/show/6192.Disgr...,"[Fiction, Africa, South Africa, Literature, No..."


# Tokenize the description using spaCy

In [4]:
import spacy
import en_core_web_md
import ast
nlp = en_core_web_md.load()
remove_tag= ['ADV','PRON','CCONJ','PUNCT','PART','DET','ADP','SPACE', 'NUM', 'SYM'] # source: suggested by chatgpt
nlp.Defaults.stop_words |= {"the","so","good","book","author","life","novel","reader","edition","writer","write","new","world","story","year"}

#tokenize descriptions
tokens = []

#pretrained en_core_web_md model for token words
for description in nlp.pipe(df["Description"]):
    tokenized_description = []
    #keep only desired tokens
    for token in description:
        #select adequate tags, skip stopwords and skip non-alphabetic
        if token.pos_ not in remove_tag and not token.is_stop and token.is_alpha:
            #use the lower case lemma for token
            tokenized_description.append(token.lemma_.lower())
    tokens.append(tokenized_description)

df['tokens'] = tokens
display(df['tokens'])

0      [famous, man, progress, search, salvation, rem...
1      [daniel, defoe, relate, tale, english, sailor,...
2      [clever, satire, use, comic, inversion, offer,...
3      [grip, perfect, fan, sadie, girl, guide, murde...
4      [foundling, mysterious, parentage, bring, allw...
                             ...                        
97     [maggie, ira, moran, marry, year, show, quarre...
98     [moran, old, republican, transform, day, glory...
99     [power, inhuman, passion, immortal, ashlyn, da...
100    [set, post, apartheid, south, africa, coetzee,...
101    [lose, father, yr, age, know, raise, lie, sile...
Name: tokens, Length: 102, dtype: object

In [5]:
# Import the necessary module from Gensim
from gensim.corpora.dictionary import Dictionary

# Create a dictionary object from the tokenized descriptions
# Each token is assigned a unique ID and the frequency of occurrence is tracked
dictionary = Dictionary(df['tokens'])

# Filter the dictionary to remove infrequent and overly common tokens
# The parameters specify the filtering criteria
# - no_below: Remove tokens that appear in fewer than 5 books/documents
# - no_above: Remove tokens that appear in more than 50% of the books/documents
# - keep_n: Keep the top 1000 most frequent tokens in the dictionary
dictionary.filter_extremes(no_below=5, no_above=0.5, keep_n=1000)

# Create a corpus representation of the tokenized descriptions using the filtered dictionary
# The corpus is a list of bag-of-words vectors, with each vector representing a document
corpus = [dictionary.doc2bow(doc) for doc in df['tokens']]


In [6]:
# Import the necessary modules for visualization, LDA modeling, and coherence scoring
import pyLDAvis
import pyLDAvis.gensim_models
pyLDAvis.enable_notebook()
from gensim.models import LdaMulticore
from gensim.models import CoherenceModel
import matplotlib as plt
# Create empty lists to store the number of topics and coherence scores
topics = []
score = []

# Iterate over a range of values from 1 to 9 (exclusive) with a step size of 1
for i in range(1, 10, 1):
    # Train an LDA model using LdaMulticore
    lda_model = LdaMulticore(corpus=corpus, id2word=dictionary, iterations=10, num_topics=i, workers=14, passes=10, random_state=100)
    
    # Calculate the coherence score using CoherenceModel
    cm = CoherenceModel(model=lda_model, texts=df['tokens'], corpus=corpus, dictionary=dictionary, coherence='c_v')
    
    # Append the current number of topics and coherence score to the respective lists
    topics.append(i)
    score.append(cm.get_coherence())

# Plot the number of topics on the x-axis and the coherence score on the y-axis
plt.plot(topics, score)

# Add labels to the x-axis and y-axis
plt.xlabel('Number of Topics')
plt.ylabel('Coherence Score')

# Display the plot
plt.show()


C:\Users\weich\Miniconda3\lib\site-packages\scipy\sparse\_sputils.py:44: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  upcast = np.find_common_type(args, [])
C:\Users\weich\Miniconda3\lib\site-packages\scipy\sparse\_sputils.py:44: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  upcast = np.find_common_type(args, [])
C:\Users\weich\Miniconda3\lib\site-packages\scipy\sparse\_sputils.py:44: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  upcast = np.find_common_type

NameError: name 'plt' is not defined

The higher the coherence score the better, in this case I decide to go with 6 topics

training an LDA model with the following parameters:
- corpus: The corpus of bag-of-words vectors.
- id2word: The dictionary mapping word IDs to words.
- iterations: The number of iterations for model training (set to 100).
- num_topics: The number of topics to be generated by the model (set to 6).
- workers: The number of core for training (set to 14).
- passes: The number of passes over the corpus during training (set to 100).

In [ ]:
lda_model = LdaMulticore(corpus=corpus, id2word=dictionary, iterations=100, num_topics= 6 , workers = 14 , passes=100)
#print all object
lda_model.print_topics(-1)

Visulize the topics' distribution and the most relevant terms for each topic.
PC1 and PC2 represent the first two principal components obtained from PCA(summary indices). The distance between points indicates the similarity or dissimilarity between topics. For example, the position of the topics on the plot is determined by their distribution of words and how they relate to each other in terms of their probabilities. Topics that are similar or have overlapping word distributions will be closer to each other on the plot, while dissimilar topics will be farther apart.

In [ ]:
lda_display = pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary)
pyLDAvis.display(lda_display)

Association with topics and genres?

In [ ]:
# Get the topic-word matrix from the LDA model
topic_word_matrix = lda_model.show_topics(formatted=False)

# Iterate over each topic and its associated words
for topic_id, topic_words in topic_word_matrix:
    print(f"Topic ID: {topic_id}")
    words = [word for word, _ in topic_words]
    print(f"Words: {', '.join(words)}\n")

In [ ]:
# Flatten the 'Genres' column
all_genres = [genre for genres_list in df['Genres'] for genre in genres_list]

# Get the unique genres
unique_genres = set(all_genres)

# Print the unique genres
print(unique_genres)
